In [1]:
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
from geopy.geocoders import Nominatim

## Geotagging the scraped Data

In [2]:
df = pd.read_csv('all_reviews.csv')
locations_df = pd.read_csv('locations_geotagged.csv')
beer_df = pd.read_csv('beer_deets.csv')

In [3]:
df

,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit
...,...,...,...,...,...,...
52344,Evil Twin Sanguinem Aurantiaco,1.6,-1.7,Evil Twin Brewing,"When I am king, citrus adjuncts will be outlaw...",BreweryChalice55
52345,Sweetwater 420 Strain Mango Kush,1.4,-1.8,Sweetwater Brewing Company,"UGH! Horrid stench of weed, skunk behind, and...",kdm1984
52346,Revolution Every Day Hero,1.8,-1.6,"Revolution Brewing Company (IL, USA)","I'm so tired of these hero beers, they all pre...",cheap
52347,Manns Brown Ale,1.0,-1.8,Marston's (Carlsberg Marston's Brewing Co.),What a disaster. Watery toffee taste. Is this ...,schenk


In [4]:
locations_df

,locations,latitude,longitude,country
0,"Bungay, Suffolk",52.456225,1.437341,United Kingdom
1,"Faversham, Kent",51.314409,0.891189,United Kingdom
2,"Bristol, Gloucestershire",51.852767,-2.255369,United Kingdom
3,"Round Rock, Texas",30.508235,-97.678893,United States of America
4,"Brooklyn, New York",40.650104,-73.949582,United States of America
...,...,...,...,...
3111,"Dentlein am Forst, Bavaria",49.147567,10.423854,Germany
3112,"Sault Ste. Marie, Ontario",46.523910,-84.320068,Canada
3113,"Bochum, North Rhine-Westphalia",51.481811,7.219663,Germany
3114,"Barth, Mecklenburg-Vorpommern",54.369055,12.725859,Germany


In [5]:
geolocator = Nominatim(user_agent="None")

In [6]:
locations_new = pd.DataFrame(df.brewery_location[~df.brewery_location.isin(locations_df.locations)].unique(),columns=['locations'])
print(len(locations_new), "new locations to search for! Will take", len(locations_new)/2, "seconds to run.")

76 new locations to search for! Will take 38.0 seconds to run.


In [7]:
def tryconvert(x):
        try:
            return geolocator.geocode(x).latitude
        except:
            return None
    

In [8]:
def tryconvertlong(x):
        try:
            return geolocator.geocode(x).longitude
        except:
            return None

In [9]:
locations_new["latitude"] = locations_new.locations.apply(lambda x: tryconvert(x))

In [10]:
locations_new["longitude"] = locations_new.locations.apply(lambda x: tryconvertlong(x))

In [11]:
countries = []
for row in locations_new.index:
    try:
        loc = geolocator.reverse([locations_new.iloc[row,1],locations_new.iloc[row,2]], language='en')
        countries.append(loc.raw['address']['country'])
    except:
        countries.append(None)
        
    

In [12]:
locations_new['country'] = countries

In [13]:
locations_new

,locations,latitude,longitude,country
0,"Market Drayton, Shropshire",52.904242,-2.483859,United Kingdom
1,"Knaresborough, North Yorkshire",54.009982,-1.462186,United Kingdom
2,"North Battleford, Saskatchewan",52.776186,-108.300476,Canada
3,"Darby, Montana",46.022659,-114.178093,United States of America
4,"Tunbridge, Vermont",43.888854,-72.493724,United States of America
...,...,...,...,...
71,"Long Branch , New Jersey",40.304278,-73.992360,United States of America
72,"Immendingen-Zimmern, Baden-Württemberg",47.932614,8.712978,Germany
73,"Tuttlingen, Baden-Württemberg",47.984431,8.818661,Germany
74,"Darwen, Lancashire",53.696096,-2.466509,United Kingdom


In [14]:
locations_df = pd.concat([locations_df,locations_new])
locations_df.to_csv('locations_geotagged.csv',index=False)

## Merging and cleaning

In [15]:
beer_df

,return_string,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number,name_found
0,St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...,🇬🇧,Bitter - Ordinary / Best,4.3,49,95,3.25,415.0,St. Peter's Ruby Red Ale
1,Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...,🇬🇧,Bitter - Ordinary / Best,4.5,34,59,3.05,1083.0,Shepherd Neame Spitfire (Bottle)
2,Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...,🇬🇧,IPA,6.0,95,97,3.74,59.0,Arbor Rocketman
3,Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...,🇬🇧,IPA - English,6.1,46,47,3.22,420.0,Shepherd Neame India Pale Ale (Bottle)
4,Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...,🇬🇧,Strong Ale - English,7.0,45,46,3.18,409.0,Shepherd Neame Christmas Ale
...,...,...,...,...,...,...,...,...,...
26756,Helicon German Pale Ale\n🇺🇸 Pale Ale\n-\n-\n2....,🇺🇸,Pale Ale,NaN,-,-,2.60,1.0,Helicon German Pale Ale
26757,Spreewalder Dunkel\n🇩🇪 Dark Lager - Dunkel / T...,🇩🇪,Dark Lager - Dunkel / Tmavý,0.0,-,-,3.42,8.0,Spreewalder Dunkel
26758,Des Cantons Allez Pou-Pou\n🇨🇦 ISA - Session IP...,🇨🇦,ISA - Session IPA,NaN,-,-,3.60,3.0,Des Cantons Allez Pou-Pou
26759,Farrenbräu Sommerpils\n🇩🇪 Pilsener / Pils / Pi...,🇩🇪,Pilsener / Pils / Pilsner,NaN,-,-,3.20,1.0,Farrenbräu Sommerpils


In [16]:
# deal with the price issue
price_df = beer_df[beer_df.rb_overall_score.str.contains('Available')].copy()
beer_df = beer_df[~beer_df.rb_overall_score.str.contains('Available')]
price_df['rb_overall_score'] = price_df.return_string.apply(lambda x: x.split('\n')[3])
price_df['rb_style_score'] = price_df.return_string.apply(lambda x: x.split('\n')[4])
price_df['rb_user_rating'] = price_df.return_string.apply(lambda x: x.split('\n')[5])
price_df['rb_rating_number'] = price_df.return_string.apply(lambda x: x.split('\n')[6].replace("(","").replace(")",""))
beer_df = pd.concat([beer_df,price_df])
print("Number of issue beers sorted:", len(price_df))


Number of issue beers sorted: 344


In [17]:
# set the types to float (cant do for the overall score and rating as there are two many None "-")
beer_df.rb_user_rating = beer_df.rb_user_rating.astype(float)
beer_df.rb_rating_number = beer_df.rb_rating_number.astype(float)

In [18]:
# Creating DF with all details
df_all = pd.merge(df,locations_df,how='left',left_on='brewery_location',right_on='locations')
df_all = pd.merge(df_all, beer_df, how='left', left_on='beer_name',right_on='name_found')
df_all


,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name,locations,latitude,longitude,country,return_string,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number,name_found
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit,"Bungay, Suffolk",52.456225,1.437341,United Kingdom,St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...,🇬🇧,Bitter - Ordinary / Best,4.3,49,95,3.25,415.0,St. Peter's Ruby Red Ale
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray,"Faversham, Kent",51.314409,0.891189,United Kingdom,Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...,🇬🇧,Bitter - Ordinary / Best,4.5,34,59,3.05,1083.0,Shepherd Neame Spitfire (Bottle)
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG,"Bristol, Gloucestershire",51.852767,-2.255369,United Kingdom,Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...,🇬🇧,IPA,6.0,95,97,3.74,59.0,Arbor Rocketman
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit,"Faversham, Kent",51.314409,0.891189,United Kingdom,Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...,🇬🇧,IPA - English,6.1,46,47,3.22,420.0,Shepherd Neame India Pale Ale (Bottle)
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit,"Faversham, Kent",51.314409,0.891189,United Kingdom,Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...,🇬🇧,Strong Ale - English,7.0,45,46,3.18,409.0,Shepherd Neame Christmas Ale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52842,Evil Twin Sanguinem Aurantiaco,1.6,-1.7,Evil Twin Brewing,"When I am king, citrus adjuncts will be outlaw...",BreweryChalice55,-1.7,48.780313,10.039726,Germany,Evil Twin Sanguinem Aurantiaco\n🇺🇸 Sour / Wild...,🇺🇸,Sour / Wild Beer - Flavored,3.3,66,21,3.27,173.0,Evil Twin Sanguinem Aurantiaco
52843,Sweetwater 420 Strain Mango Kush,1.4,-1.8,Sweetwater Brewing Company,"UGH! Horrid stench of weed, skunk behind, and...",kdm1984,-1.8,48.780981,10.037026,Germany,Sweetwater 420 Strain Mango Kush\n🇺🇸 Wheat Ale...,🇺🇸,Wheat Ale,5.0,47,80,3.21,68.0,Sweetwater 420 Strain Mango Kush
52844,Revolution Every Day Hero,1.8,-1.6,"Revolution Brewing Company (IL, USA)","I'm so tired of these hero beers, they all pre...",cheap,-1.6,48.718182,9.989251,Germany,Revolution Every Day Hero\n🇺🇸 ISA - Session IP...,🇺🇸,ISA - Session IPA,4.3,60,52,3.26,11.0,Revolution Every Day Hero
52845,Manns Brown Ale,1.0,-1.8,Marston's (Carlsberg Marston's Brewing Co.),What a disaster. Watery toffee taste. Is this ...,schenk,-1.8,48.780981,10.037026,Germany,Manns Brown Ale\n🇬🇧 Mild Ale • 2.8%\n16\n8\n2....,🇬🇧,Mild Ale,2.8,16,8,2.77,194.0,Manns Brown Ale


In [19]:
# drop excess columns
df_all.drop(["locations","return_string","name_found"],axis=1,inplace=True)
# drop rows with ratings instead of location
print(len(df_all[df_all['brewery_location'].str.len()<4]),"columns without locations dropped.")
df_all = df_all[~df_all['brewery_location'].str.len()<4]
# drop all NaNs
print(len(df_all) - len(df_all.dropna()), "columns with NaNs dropped.")
df_all.dropna(inplace=True)


238 columns without locations dropped.
4176 columns with NaNs dropped.


In [20]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48671 entries, 0 to 52846
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   beer_name         48671 non-null  object 
 1   beer_rating       48671 non-null  float64
 2   brewery_location  48671 non-null  object 
 3   brewery_name      48671 non-null  object 
 4   review_content    48671 non-null  object 
 5   reviewer_name     48671 non-null  object 
 6   latitude          48671 non-null  float64
 7   longitude         48671 non-null  float64
 8   country           48671 non-null  object 
 9   flag              48671 non-null  object 
 10  type              48671 non-null  object 
 11  abv               48671 non-null  float64
 12  rb_overall_score  48671 non-null  object 
 13  rb_style_score    48671 non-null  object 
 14  rb_user_rating    48671 non-null  float64
 15  rb_rating_number  48671 non-null  float64
dtypes: float64(6), object(10)
memory usage: 

## Deal with Non-English Descriptions

## Save down cleaned data

In [21]:
df_all.to_csv("data_clean.csv",index=False)